## Notes 
### Différents type de régimes : 
Foodmap : Favorise certains aliments contenant des glucides capable de fermenter dans l'intestin. 
Méditerranéen : Favorise l'équilibre alimentaire, pour privilégier la variété des aliments. 
Dash (Dietary Approaches to Stopping Hypertension) : Permet de combattre l'hypertension artérielle en modifiant les habitudes alimentaires.
À indice glycémique bas : Régule le taux de sucre dans le sang en fonction des aliments et deux leur indice glycémique (IG).
Végétarien/Végétalisme : Exclut toutes chaires animales / Exclut la consommation de tous produits d'origine animale.
Paléolithique : Favorise la même alimentation que nos ancêtres, avant l'ère industrielle. 
Cétogène : Suppresion quasi complète des sucres et amidons(glucides).

### Menus en fonction des régimes : 
Paléolithique : https://www.passeportsante.net/fr/Nutrition/Regimes/Fiche.aspx?doc=paleolithique_regime

## Étapes d'avancements 
Étape 1 : Nettoyage des données<br>
Étape 2 : Importer les données dans une base de donnée <br>
Étape 3 : Création de menu, et de profil d'utilisateur (catégorie)<br>
Étape 4 : Importer ces nouvelles données dans la base de donnée <br>
Étape 5 : Mettre en corrélation les aliments avec les menus & utilisateur <br>

In [41]:
# Import nécessaires des packages 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

In [42]:
# Création d'une session Spark, entrée principale pour intéragir avec pyspark
spark = SparkSession.builder.appName("data_integration_openfoodfacts").getOrCreate()

# Importation des données openfoodfacts dans le dataframe, délimitation sur la tabulation et récupération de l'en-tête 
df_openfoodfacts = spark.read.option("delimiter", "\t").csv("../data/en.openfoodfacts.org.products.csv", header=True, inferSchema=True)

In [11]:
# Affiche le schéma du dataframe, indiquant les colonnes et leurs types de données 
df_openfoodfacts.printSchema()

root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: integer (nullable = true)
 |-- last_updated_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- 

### <b>Étape 2</b> : Nettoyage des données importées 
<hr/>

#### Suppression des valeurs nulles

In [43]:
# Supprime les lignes ayant des valeurs nulles 
df_openfoodfacts.dropna()

3085838

#### Suppresion des valeurs aberrantes

<b>Énergie (kcal/100g)</b> : Les valeurs caloriques par portion devraient se situer entre 0 kcal/100g (minimum) et 800 kcal/100g (maximum) pour être considérées comme non aberrantes.

<b>Protéines (g/100g)</b> : La teneur en protéines par portion serait considérée comme non aberrante si elle se situe dans la plage de 1 g/100g (minimum) à 100 g/100g (maximum).

<b>Graisses (g/100g)</b> : Les quantités de graisses par portion devraient être comprises entre un minimum de 0 g/100g et un maximum de 100 g/100g pour être considérées comme non aberrantes.

<b>Glucides (g/100g)</b> : La teneur en glucides par portion serait jugée non aberrante si elle se situe entre 1 g/100g (minimum) et 80 g/100g (maximum).

In [52]:
# Définir des valeurs spécifiques pour chaque colonne
valeurs_specifiques = {
    'energy-kcal_100g': (0, 800),
    'proteins_100g': (1, 100),
    'fat_100g': (0, 100),
    'carbohydrates_100g': (1, 80)
}

# Filtrer les produits qui contiennent des valeurs aberrantes pour les informations nutritionnelles
df_openfoodfacts = df_openfoodfacts.filter(
    (col("energy-kcal_100g") > valeurs_specifiques['energy-kcal_100g'][0]) &
    (col("energy-kcal_100g") <= valeurs_specifiques['energy-kcal_100g'][1]) &
    (col("proteins_100g") > valeurs_specifiques['proteins_100g'][0]) &
    (col("proteins_100g") <= valeurs_specifiques['proteins_100g'][1]) &
    (col("fat_100g") > valeurs_specifiques['fat_100g'][0]) &
    (col("fat_100g") <= valeurs_specifiques['fat_100g'][1]) &
    (col("carbohydrates_100g") > valeurs_specifiques['carbohydrates_100g'][0]) &
    (col("carbohydrates_100g") <= valeurs_specifiques['carbohydrates_100g'][1])
)

# Afficher le nombre total de lignes après avoir enlevé les valeurs aberrantes
print(df_openfoodfacts.count())

1194795


In [6]:
# Arrête la session Spark 
spark.stop()